In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
import os

Mounted at /content/gdrive


CONSUMER_KEY: cPOXRburCBWOBLNTkPpcnfS4f

---


CONSUMER_SECRET_KEY: jxW01PDcTpCaNagq6I1gJbQd7jEM2hjWW75qYKFkU7JvZrLMBl

---

ACCESS_TOKEN_KEY: 1460972075120799746-XStq5Vb1yZCvBUzvPPEnBvuZzf35wO

---

ACCESS_TOKEN_SECRET_KEY: vznwLLjlnrluMfMlCl4YbWjX5BriLRtTwH3xYjDN8Y0ef

---

Bearer Token : AAAAAAAAAAAAAAAAAAAAAPmTVwEAAAAA%2Fa%2FcvId%2FY%2BKPsV1FCLdM2F47mJg%3DbLsf0JCPFVD7VV8ygdjbZLl5cvOKSSe0urgKL48MUfkWnZY3jN


In [ ]:
!pip install twarc
!twarc configure

In [10]:

!pip install argparse #Argparse 3.2
!pip install xtract #Xtract 0.1 a3
!pip install wget #Wget 3.2


# import pycountry
import tweepy
import pandas as pd
import requests
import gzip
import os
import wget
import csv
import linecache
from shutil import copyfile
import ipywidgets as widgets
import numpy as np
from tqdm.auto import tqdm
from twarc import Twarc



  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


ModuleNotFoundError: ignored

In [11]:

import shutil

dates = open("/content/gdrive/MyDrive/Colab/COMS6998EMODS/dates.txt").readlines()


id_dir = "/content/gdrive/MyDrive/Colab/COMS6998EMODS/waste/"
url_dir = 'https://github.com/thepanacealab/covid19_twitter/blob/master/dailies/' 
#dataset_URL = "https://github.com/thepanacealab/covid19_twitter/blob/master/dailies/2021-01-20/2021-01-20_clean-dataset.tsv.gz?raw=true"

cnt = 0
total_cnt = 0

for date in tqdm(dates):
  date = date.split('\n')[0]
  url = os.path.join(url_dir, date, date+"_clean-dataset.tsv.gz?raw=true")
  id_filename = os.path.join(id_dir, date+"_clean-dataset.tsv")

 
  wget.download(url, out='clean-dataset.tsv.gz')

  #Unzips the dataset and gets the TSV dataset
  with gzip.open('clean-dataset.tsv.gz', 'rb') as f_in:
      with open(id_filename, 'wb') as f_out:
          shutil.copyfileobj(f_in, f_out)

  #Deletes the compressed GZ file
  os.unlink("clean-dataset.tsv.gz")

  df = pd.read_csv(id_filename, sep='\t', index_col=0)
  
  #Deletes the 2020-03-24.tsv file
  os.unlink(id_filename)

  total_cnt += df.shape[0]


  # if  'country_code' not in df.columns.values.tolist():
  #   print('No Lang or Country', date)
    
  # else:

  #   df = df[df['lang'] == 'en']
  #   df = df[df['country_code'] == 'US']

  #   df.reset_index(inplace=True)
  #   print(date, df.shape)

  #   src_file = id_dir + date + '_ids.txt'
  #   ids = list(df['tweet_id'])
  #   ids = pd.DataFrame(ids, columns=["id"])
  #   ids.to_csv(src_file, index=False, header = False)

total_cnt

  0%|          | 0/153 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


83624190

In [ ]:
import os
import json
import glob
from tqdm.auto import tqdm


HOME_DIR = "/content/gdrive/MyDrive/Colab/COMS6998EMODS/Oct_hydrate/ID/*"
DEST_DIR = "/content/gdrive/MyDrive/Colab/COMS6998EMODS/Oct_hydrate/Tweet/"

dest_files = glob.glob(DEST_DIR + "*")

for fname in tqdm(glob.glob(HOME_DIR)):
  name = fname.split('/')[-1].split('_')[0] + ".json"
  output_file = DEST_DIR + name
  if output_file not in dest_files:
    # print(output_file)
    cmd = "twarc hydrate " + str(fname) + " > " + str(output_file)
    os.system(cmd)
print('Hydrate Done.')


  0%|          | 0/31 [00:00<?, ?it/s]

Hydrate Done.


In [ ]:
import os
import json
import glob
import pandas as pd
from tqdm.auto import tqdm

HOME_DIR = "/content/gdrive/MyDrive/Colab/COMS6998EMODS/json_Files/Dec_2020/*"
DEST_DIR = "/content/gdrive/MyDrive/Colab/COMS6998EMODS/Hydrated_Tweets_All_Attributes/Dec_2020/"

df = pd.read_csv('/content/gdrive/MyDrive/Colab/COMS6998EMODS/state_code.csv', sep=',')
state2code = {}
for index, row in df.iterrows():
  state = row['name'].lower()
  code = row['code']
  state2code[state] = code


FILES = glob.glob(HOME_DIR)

cnt = 0
for fname in tqdm(FILES):

  name = fname.split('/')[-1].split('.')[0] + ".csv"
  data = pd.read_json(fname, lines=True)
  # print(data.columns)


  # data = data.drop(['created_at', 'id_str', 'truncated', 'display_text_range', 'entities', 'in_reply_to_status_id', 'source', \
  #                   'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', \
  #                   'geo', 'coordinates', 'retweet_count', 'favorite_count',  'favorited', 'retweeted', \
  #                   'contributors', 'is_quote_status', 'possibly_sensitive', 'lang', 'extended_entities', 'quoted_status_id', \
  #                   'quoted_status_id_str', 'quoted_status_permalink', 'quoted_status'], axis=1)
  
  data = data[data['full_text'].notna()]
  data = data[data['place'].notna()]


  states = []
  drop_index = []
  for index, row in data.iterrows():
    place = row['place']
    place_type = place['place_type']
      
    if place_type == 'admin':
      state, country = place['full_name'].split(',')
      code = state2code[state.lower()]
      states.append(code)
    elif place_type == 'city':
      city, state = place['full_name'].split(',')
      states.append(state)
    else:
      drop_index.append(index)


  data = data.drop(index=drop_index)
  data['state'] = states
  cnt += data.shape[0]
  # print(data.shape)
  data.to_csv(DEST_DIR + "/" + name, index=False)
print(cnt)

  0%|          | 0/31 [00:00<?, ?it/s]

106091
